In [ ]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch import multiprocessing
from torch import nn
from tensordict.nn import TensorDictModule

from collections import defaultdict

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch import multiprocessing

In [ ]:
is_fork = multiprocessing.get_start_method() == "fork"

device = (
    torch.device(0)
    if torch.cuda.is_available() and not is_fork
    else torch.device("cpu")
)

## Environment Preparation

#### Load unity environment using `mlagents_envs`

In [ ]:
from mlagents_envs.side_channel.engine_configuration_channel import EngineConfigurationChannel
from mlagents_envs.environment import UnityEnvironment

channel = EngineConfigurationChannel()
env_path = "C:/Users/Pawel/Documents/Unity_Project/warehouse-bot-training/environment_builds/test_env_simplified/Warehouse_Bot.exe"

from torchrl.envs import UnityMLAgentsEnv

unity_env = UnityEnvironment(
  file_name=env_path,
  side_channels=[channel],
  additional_args=["-batchmode", "-nographics"]
)
channel.set_configuration_parameters(time_scale=10)

#### Transform environment from `mlagents` to `gymnasium`

In [ ]:
import gymnasium as gym

In [ ]:
print(gym.__version__)

In [ ]:
from env_gymnasium_wrapper import UnityGymWrapper

gymnasium_env = UnityGymWrapper(unity_env)

#### Creating stable_baselines3 model

In [ ]:
# from stable_baselines3.common.env_util import make_vec_env
# from stable_baselines3.common.vec_env import SubprocVecEnv

# env = make_vec_env(gymnasium_env, n_envs=8, vec_env_cls=SubprocVecEnv)
# model = PPO("MlpPolicy", env, device="cpu")
# model.learn(total_timesteps=25_000)

In [ ]:
from stable_baselines3 import PPO

model = PPO("MlpPolicy", gymnasium_env, verbose=1,
            learning_rate=3e-4,
            n_steps=10240,
            batch_size=256,
            n_epochs=6,
            clip_range=0.2,
            gamma=0.99,
            gae_lambda=0.96,
            seed=0,
            ent_coef=0.005,
            vf_coef=0.5,
            
            
)

In [ ]:
model.learn(total_timesteps=20_000)

In [ ]:
model.save('./saved_models/test_2')

In [ ]:
model2 = PPO.load("./saved_models/test_2", gymnasium_env)

In [ ]:
model2.learn(total_timesteps=100_000)